# Functions

In [ ]:
import torch
import seaborn as sns
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import scipy
from sklearn import metrics
from scipy.stats import pearsonr
#from ROIPredictor import RoiPredictor
from torch import nn
%matplotlib inline

In [ ]:
BASE = '/home/guests/paul_hager/Projects/ATN_Classification/'

In [ ]:
def plotAllPred(df, target):
    sns.set(font_scale=1.5)
    g = sns.displot(data = df, x='{} GT ROI SUVR'.format(target), y='{} Pred ROI SUVR'.format(target), height=10, aspect=1.5, cbar=True)
    max_val = max(max(df['{} GT ROI SUVR'.format(target)].values),max(df['{} Pred ROI SUVR'.format(target)].values))
    g.set_axis_labels('Amyloid SUVR GT','Amyloid SUVR Prediction')
    g.set(ylim=(0,max_val))
    g.set(xlim=(0,max_val))
    plt.plot([0,max_val],[0,max_val], linewidth=1, color='red')
    #plt.show()

In [ ]:
def plotAllPred_density(df, target):
    sns.set(font_scale=1.5)
    g = sns.displot(data = df, x='{} GT ROI SUVR'.format(target), y='{} Pred ROI SUVR'.format(target), height=10, aspect=1.5, cbar=True)
    max_val = max(max(df['{} GT ROI SUVR'.format(target)].values),max(df['{} Pred ROI SUVR'.format(target)].values))
    g.set_axis_labels('Gray Matter Density GT','Gray Matter Density Prediction')
    g.set(ylim=(0.4,max_val))
    g.set(xlim=(0.4,max_val))
    plt.plot([0,max_val],[0,max_val], linewidth=1, color='red')
    #plt.show()

In [ ]:
def Calc_FBB_From_AV45(AV45_SUVR):
    return (1.2377*AV45_SUVR) - 0.279

In [ ]:
def Calc_Global_SUVR(row, target):
    avg_suvr = 0
    for i in range(1,201):
        avg_suvr += row['{} ROI {}'.format(target,i)]
    avg_suvr = avg_suvr / 200
    return avg_suvr

In [ ]:
def Calc_Global_SUVR_dkt(row, target):
    avg_suvr = 0
    for i in dktLabels:
        avg_suvr += row['{} ROI {}'.format(target,i)]
    avg_suvr = avg_suvr / len(dktLabels)
    return avg_suvr

In [ ]:
def Calc_Abeta_SUVR(row, target):
    avg_suvr = 0
    for i in abetaDKTLabels:
        avg_suvr += row['{} ROI {}'.format(target,i)]
    avg_suvr = avg_suvr / len(abetaDKTLabels)
    return avg_suvr

In [ ]:
def calc_total_vol(row):
    regions = [4001, 4002, 4011, 4012, 4021, 4022, 4101, 4102, 4111, 4112, 4201, 4202] # Entire medial temporal lobe
    regions = [4101,4102] # Hippocampus only
    total = 0
    for l in regions:
        total += row['GM Volume ROI {}'.format(l)]
    return total

In [ ]:
cutoff_abeta_av45 = 1.11
cutoff_tau = 1.3



commonLabels = [2001, 2002, 2101, 2102, 2111, 2112, 2201, 2202, 2211, 2212, 2301, 2302, 2311, 2312, 2321, 2322, 2331, 2332, 2401, 2402, 2501, 2502, 2601, 2602, 2611, 2612, 2701, 2702, 3001, 3002, 4001, 4002, 4011, 4012, 4021, 4022, 4101, 4102, 4111, 4112, 4201, 4202, 5001, 5002, 5011, 5012, 5021, 5022, 5101, 5102, 5201, 5202, 5301, 5302, 5401, 5402, 6001, 6002, 6101, 6102, 6201, 6202, 6211, 6212, 6221, 6222, 6301, 6302, 6401, 6402, 7001, 7002, 7011, 7012, 7021, 7022, 7101, 7102, 8101, 8102, 8111, 8112, 8121, 8122, 8201, 8202, 8211, 8212, 8301, 8302, 9001, 9002, 9022, 9031, 9032, 9041, 9042, 9120]
dktLabels = [2,4,5,7,8,10,11,12,13,14,15,16,17,18,24,26,28,30,31,41,43,44,46,47,49,50,51,52,53,54,58,60,62,63,77,85,251,252,253,254,255,1000,1002,1003,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1034,1035,2000,2002,2003,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2034,2035]
abetaDKTLabels = [1003, 1012, 1014, 1018, 1019, 1020, 
    1027, 1028, 2003, 2012, 2014, 2018, 
    2019, 2020, 2027, 2028, 1002, 1010, 
    1023, 1026, 2002, 2010, 2023, 2026, 
    1008, 1025, 1029, 1031, 2008, 2025, 
    2029, 2031, 1015, 1030, 2015, 2030]

In [ ]:
len(commonLabels)

# Eval - CSV

## GM Volume

### Binary

In [ ]:
name = 'roi_predictor/csv/binary/gmvolume/version_13'
splits = 'splits_volume'
target = 'gmvolume'

all_results_df = pd.DataFrame(columns=['{} GT'.format(target), '{} Pred'.format(target)])

for i in range(5,6):
    gt = 'split_{}'.format(i)
    val_f = f'{BASE}/data/{}/split_{}.csv'.format(splits,i) # MAKE SURE CORRECT SPLIT
    preds_f = f'{BASE}/tb_logs/{}/{}/preds.csv'.format(name,gt)

    with open(val_f, "r") as f:
        gts = f.readlines()
    gt_target = [int(x.strip().split(',')[-1]) for x in gts]
    
    with open(preds_f, "r") as f:
        preds = f.readlines()
    preds = [int(x.strip()) for x in preds]
    
    results_df = pd.DataFrame({'{} GT'.format(target):gt_target, '{} Pred'.format(target):preds})
    
    all_results_df = pd.concat([all_results_df,results_df])

    print(metrics.f1_score(results_df['{} GT'.format(target)], results_df['{} Pred'.format(target)]))

### ROI

In [ ]:
name = 'roi_predictor/csv/roi/gmvolume/version_22'
target = 'gmvolume'
nontarget = 'Tau'
splits = 'splits_volume_ATN_schaefer'
all_results_df = pd.DataFrame(columns = ['{} GT ROI SUVR'.format(target),'{} Pred ROI SUVR'.format(target),'{} GT ROI SUVR'.format(nontarget)])

gt_hip_vols = []
pred_hip_vols = []

for i in range(5,6):
    gt = 'split_{}'.format(i)
    val_f = f'{BASE}/data/{}/{}.csv'.format(splits,gt) # MAKE SURE CORRECT SPLIT
    preds_f = f'{BASE}/tb_logs/{}/{}/preds.csv'.format(name,gt)

    header_gt = ['Female','Male','Age','33','34','44','24','23','22']
    header_gt.extend(['Tau ROI {}'.format(x) for x in range(1,201)])
    #header_gt.extend(['Tau ROI {}'.format(x) for x in dktLabels])
    header_gt.extend(['GM Volume ROI {}'.format(x) for x in commonLabels])
    header_preds = ['GM Volume ROI {}'.format(x) for x in commonLabels]

    gt_df = pd.read_csv(val_f, names=header_gt)
    pred_df = pd.read_csv(preds_f, names=header_preds)
    
    #gt_df = gt_df[(gt_df.iloc[:,0]=='GM Volume ROI 4101')|(gt_df.iloc[:,0]=='GM Volume ROI 4102')]
    #pred_df = pred_df[(pred_df.iloc[:,0]=='GM Volume ROI 4101')|(pred_df.iloc[:,0]=='GM Volume ROI 4102')]
    
    gt_df['Hip Volume'] = gt_df.apply(lambda row: calc_total_vol(row),axis=1)
    pred_df['Hip Volume'] = pred_df.apply(lambda row: calc_total_vol(row),axis=1)
    gt_hip_vols.extend(gt_df['Hip Volume'])
    pred_hip_vols.extend(pred_df['Hip Volume'])

    gt_roi_melt_df = pd.melt(gt_df, value_vars = header_preds, value_name='{} GT ROI SUVR'.format(target))
    pred_roi_melt_df = pd.melt(pred_df, value_vars = header_preds, value_name='{} Pred ROI SUVR'.format(target))
    results_df = pd.concat([gt_roi_melt_df,pred_roi_melt_df],axis=1)
    all_results_df = pd.concat([all_results_df,results_df[['{} GT ROI SUVR'.format(target),'{} Pred ROI SUVR'.format(target)]]])

    plotAllPred(results_df, target)
    outp = os.path.join(os.path.dirname(preds_f),'density.jpg')
    plt.savefig(outp)
    
gt_df_gmdensity = gt_df.copy(deep=True)

In [ ]:
#plotAllPred(all_results_df, target)
plotAllPred_density(all_results_df, target)
r = pearsonr(all_results_df['gmvolume Pred ROI SUVR'],all_results_df['gmvolume GT ROI SUVR'])
print(r)
plt.savefig(os.path.join(f'{BASE}/tb_logs',name,'all_splits_density.jpg'))

#### Calc Summary Metrics

In [ ]:
hip_vols_df = pd.DataFrame({'GT':gt_hip_vols,'Pred':pred_hip_vols})
if True:
    cutoff = 1.514 # 1.5 std (0.09655) away from mean (1.659)
    cutoff = 1.562 # 1 std away from mean
cutoff = 1.6474876557377052 - 1 * 0.08779840023070665 # Hipp only
#cutoff = 9.938946737704917 - 1 * 0.44420289103294347 # Medial Temporal Lobe

hip_vols_df['GT Positive'] = hip_vols_df.apply(lambda row: 1 if row['GT']<cutoff else 0, axis=1)
hip_vols_df['Pred Positive'] = hip_vols_df.apply(lambda row: 1 if row['Pred']<cutoff else 0, axis=1)

In [ ]:
sum(hip_vols_df[['GT Positive']].values)/len(hip_vols_df[['GT Positive']].values)

In [ ]:
precision = metrics.precision_score(hip_vols_df['GT Positive'], hip_vols_df['Pred Positive'])
recall = metrics.recall_score(hip_vols_df['GT Positive'], hip_vols_df['Pred Positive'])
f1 = metrics.f1_score(hip_vols_df['GT Positive'], hip_vols_df['Pred Positive'])
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1: {}'.format(f1))

In [ ]:
metrics.roc_auc_score(hip_vols_df['GT Positive'], hip_vols_df['Pred Positive'])

## Amyloid

In [ ]:
name = 'roi_predictor/csv/roi/amyloid/version_18'
splits = 'splits_FBB_as_AV45/dkt'
target = 'Abeta'
nontarget = 'Tau' if target=='Abeta' else 'Abeta'
all_results_df = pd.DataFrame(columns = ['{} GT ROI SUVR'.format(target),'{} Pred ROI SUVR'.format(target),'{} GT ROI SUVR'.format(nontarget)])
all_gt_global_SUVR = []
all_gt_nontarget_global_SUVR = []
all_pred_global_SUVR = []

for i in range(5,6):
    gt = 'split_{}'.format(i)
    val_f = f'{BASE}/data/{}/{}.csv'.format(splits,gt) # MAKE SURE CORRECT SPLIT
    preds_f = f'{BASE}/tb_logs/{}/{}/preds.csv'.format(name,gt)

    header_gt = ['Sex F','Sex M','Age','33','34','44','24','23','22']
    header_gt.extend(['Tau ROI {}'.format(x) for x in dktLabels])
    header_gt.extend(['Abeta ROI {}'.format(x) for x in dktLabels])
    header_preds = ['{} ROI {}'.format(target,x) for x in dktLabels]
    header_nontarget = ['{} ROI {}'.format(nontarget,x) for x in dktLabels]

    gt_df = pd.read_csv(val_f, names=header_gt)
    pred_df = pd.read_csv(preds_f, names=header_preds)
    
    gt_df['Global {} SUVR'.format(target)] = gt_df.apply(lambda row: Calc_Abeta_SUVR(row,target), axis=1)
    gt_df['Global {} SUVR'.format(nontarget)] = gt_df.apply(lambda row: Calc_Global_SUVR_dkt(row,nontarget), axis=1)
    pred_df['Global {} SUVR'.format(target)] = pred_df.apply(lambda row: Calc_Abeta_SUVR(row,target), axis=1)
    
    all_gt_global_SUVR.extend(gt_df['Global {} SUVR'.format(target)])
    all_gt_nontarget_global_SUVR.extend(gt_df['Global {} SUVR'.format(nontarget)])
    all_pred_global_SUVR.extend(pred_df['Global {} SUVR'.format(target)])    

    gt_roi_melt_df = pd.melt(gt_df, value_vars = header_preds, value_name='{} GT ROI SUVR'.format(target))
    gt_roi_nontarget_melt_df = pd.melt(gt_df, value_vars = header_nontarget, value_name='{} GT ROI SUVR'.format(nontarget))
    pred_roi_melt_df = pd.melt(pred_df, value_vars = header_preds, value_name='{} Pred ROI SUVR'.format(target))
    results_df = pd.concat([gt_roi_melt_df,pred_roi_melt_df,gt_roi_nontarget_melt_df],axis=1)
    all_results_df = pd.concat([all_results_df,results_df[['{} GT ROI SUVR'.format(target),'{} Pred ROI SUVR'.format(target),'{} GT ROI SUVR'.format(nontarget)]]])

    plotAllPred(results_df, target)
    outp = os.path.join(os.path.dirname(preds_f),'density.jpg')
    plt.savefig(outp)

all_global_SUVR = pd.DataFrame({'{} GT ROI SUVR'.format(target):all_gt_global_SUVR,'{} Pred ROI SUVR'.format(target):all_pred_global_SUVR,'{} GT ROI SUVR'.format(nontarget):all_gt_nontarget_global_SUVR})
gt_df_amyloid = gt_df.copy(deep=True)

In [ ]:
results_df

In [ ]:
results_df.to_csv('../data/regional_amyloid_results.csv')
#all_results_df

In [ ]:
plotAllPred(all_results_df, target)
r = pearsonr(all_results_df['{} Pred ROI SUVR'.format(target)],all_results_df['{} GT ROI SUVR'.format(target)])
print(r)
plt.savefig(os.path.join(f'{BASE}/tb_logs',name,'all_splits_density.jpg'))

### Calc Summary Metrics

In [ ]:
regional = False

av45_cutoff = 1.11
fbb_cutoff = 1.08

if target == 'Abeta':
    cutoff_t = 1.11
    cutoff_nt = 1.3
else:
    cutoff = 1.3
    cutoff_nt = 1.11
        
if regional:
    bench_df = all_results_df
else:
    bench_df = all_global_SUVR
    
bench_df['{} GT Positive'.format(target)] = bench_df.apply(lambda row: 1 if row['{} GT ROI SUVR'.format(target)]>cutoff_t else 0, axis=1)
bench_df['{} Pred Positive'.format(target)] = bench_df.apply(lambda row: 1 if row['{} Pred ROI SUVR'.format(target)]>cutoff_t else 0, axis=1)
bench_df['{} GT Positive'.format(nontarget)] = bench_df.apply(lambda row: 1 if row['{} GT ROI SUVR'.format(nontarget)]>cutoff_nt else 0, axis=1)

precision = metrics.precision_score(bench_df['{} GT Positive'.format(target)], bench_df['{} Pred Positive'.format(target)])
recall = metrics.recall_score(bench_df['{} GT Positive'.format(target)], bench_df['{} Pred Positive'.format(target)])
f1 = metrics.f1_score(bench_df['{} GT Positive'.format(target)], bench_df['{} Pred Positive'.format(target)])
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1 score is: {}'.format(f1))

auc = metrics.roc_auc_score(bench_df['{} GT Positive'.format(target)], bench_df['{} Pred Positive'.format(target)])
print('AUC score is: {}'.format(auc))


x = len(bench_df.loc[(bench_df['{} GT Positive'.format(target)]==1)&(bench_df['{} Pred Positive'.format(target)]==1)&(bench_df['{} GT Positive'.format(nontarget)]==0)])
y = len(bench_df.loc[(bench_df['{} GT Positive'.format(target)]==1)&(bench_df['{} Pred Positive'.format(target)]==0)&(bench_df['{} GT Positive'.format(nontarget)]==0)])
print('Out of {} instances, {} found and {} missed. Thats {}%'.format(x+y,x,y,x/(x+y)*100))

In [ ]:
(x+y)/len(bench_df)

## Multitask

### Amyloid

In [ ]:
name = 'roi_predictor/csv/multitask/amyloid/version_9'
splits = 'splits_FBB_as_AV45_multitask/dkt'
#splits = 'splits_AV45_multitask/dkt'
target = 'Abeta'
nontarget = 'Tau'
all_results_df = pd.DataFrame(columns = ['Abeta GT ROI SUVR', 'Abeta Pred ROI SUVR', 'Tau GT ROI SUVR'])
all_gt_global_SUVR = []
all_gt_nontarget_global_SUVR = []
all_pred_global_SUVR = []
all_gt_labels = []
all_pred0 = []
all_pred1 = []

for i in range(5,6):
    gt = 'split_{}'.format(i)
    val_f = f'{BASE}/data/{splits}/{gt}.csv'  # MAKE SURE CORRECT SPLIT
    preds_f = f'{BASE}/tb_logs/{name}/{gt}/preds.csv'

    header_gt = ['Sex F','Sex M','Age','33','34','44','24','23','22']
    header_gt.extend(['Tau ROI {}'.format(x) for x in dktLabels])
    header_gt.extend(['Abeta ROI {}'.format(x) for x in dktLabels])
    header_gt.append('A+')
    header_preds = ['Pred 0','Pred 1']
    header_preds.extend(['Abeta ROI {}'.format(x) for x in dktLabels])
    header_nontarget = ['Tau ROI {}'.format(x) for x in dktLabels]

    gt_df = pd.read_csv(val_f, names=header_gt)
    pred_df = pd.read_csv(preds_f, names=header_preds)
    
    gt_df['Global Abeta SUVR'] = gt_df.apply(lambda row: Calc_Abeta_SUVR(row,target), axis=1)
    gt_df['Global Tau SUVR'] = gt_df.apply(lambda row: Calc_Global_SUVR_dkt(row,nontarget), axis=1)
    pred_df['Global Abeta SUVR'] = pred_df.apply(lambda row: Calc_Abeta_SUVR(row,target), axis=1)
    
    all_gt_global_SUVR.extend(gt_df['Global Abeta SUVR'])
    all_gt_nontarget_global_SUVR.extend(gt_df['Global Tau SUVR'])
    all_pred_global_SUVR.extend(pred_df['Global Abeta SUVR'])
    
    all_gt_labels.extend(gt_df['A+'])
    all_pred0.extend(pred_df['Pred 0'])
    all_pred1.extend(pred_df['Pred 1'])
    
    roi_names = ['Abeta ROI {}'.format(x) for x in dktLabels]

    gt_roi_melt_df = pd.melt(gt_df, value_vars = roi_names, value_name='Abeta GT ROI SUVR')
    gt_roi_nontarget_melt_df = pd.melt(gt_df, value_vars = header_nontarget, value_name='Tau GT ROI SUVR')
    pred_roi_melt_df = pd.melt(pred_df, value_vars = roi_names, value_name='Abeta Pred ROI SUVR')
    results_df = pd.concat([gt_roi_melt_df,pred_roi_melt_df,gt_roi_nontarget_melt_df],axis=1)
    all_results_df = pd.concat([all_results_df,results_df[['Abeta GT ROI SUVR', 'Abeta Pred ROI SUVR', 'Tau GT ROI SUVR']]])

    plotAllPred(results_df, target)
    outp = os.path.join(os.path.dirname(preds_f),'density.jpg')
    plt.savefig(outp)

all_global_SUVR = pd.DataFrame({'Abeta GT ROI SUVR':all_gt_global_SUVR, 'Abeta Pred ROI SUVR':all_pred_global_SUVR, 'Tau GT ROI SUVR':all_gt_nontarget_global_SUVR, 'GT Label':all_gt_labels, 'Pred0':all_pred0, 'Pred1':all_pred1})
gt_df_amyloid = gt_df.copy(deep=True)

In [ ]:
plotAllPred(all_results_df, target)
r = pearsonr(all_results_df['Abeta Pred ROI SUVR'],all_results_df['Abeta GT ROI SUVR'])
print(r)
plt.savefig(os.path.join(f'{BASE}/tb_logs',name,'all_splits_density.jpg'))

#### Calc Summary Metrics

In [ ]:
all_global_SUVR[['Pred0_sm','Pred1_sm']] = all_global_SUVR.apply(lambda row: scipy.special.softmax(row[['Pred0','Pred1']]),axis=1)
all_global_SUVR['Pred Label'] = all_global_SUVR.apply(lambda row: np.argmax(row[['Pred0_sm','Pred1_sm']]),axis=1)

In [ ]:
# Determine decision cutoff from train data
if True:
    pr, re, th = metrics.precision_recall_curve(all_global_SUVR['GT Label'],all_global_SUVR['Pred1_sm'])
    f1 = scipy.stats.hmean([pr,re])
    best_i = np.argmax(f1)
    threshold_a = th[best_i]

In [ ]:
all_global_SUVR.to_csv('../data/global_suvr_amyloid.csv')

In [ ]:
all_global_SUVR['Abeta GT Positive'] = all_global_SUVR.apply(lambda row: 1 if row['Abeta GT ROI SUVR']>cutoff_abeta_av45 else 0, axis=1)
all_global_SUVR['Abeta Pred Positive'] = all_global_SUVR.apply(lambda row: 1 if row['Abeta Pred ROI SUVR']>cutoff_abeta_av45 else 0, axis=1)
all_global_SUVR['Tau GT Positive'] = all_global_SUVR.apply(lambda row: 1 if row['Tau GT ROI SUVR']>cutoff_tau else 0, axis=1)

all_results_df['Abeta GT Positive'] = all_results_df.apply(lambda row: 1 if row['Abeta GT ROI SUVR']>cutoff_abeta_av45 else 0, axis=1)
all_results_df['Abeta Pred Positive'] = all_results_df.apply(lambda row: 1 if row['Abeta Pred ROI SUVR']>cutoff_abeta_av45 else 0, axis=1)
all_results_df['Tau GT Positive'] = all_results_df.apply(lambda row: 1 if row['Tau GT ROI SUVR']>cutoff_tau else 0, axis=1)

assert np.all(all_global_SUVR['Abeta GT Positive']==all_global_SUVR['GT Label'])

pr, re, th = metrics.precision_recall_curve(all_global_SUVR['GT Label'],all_global_SUVR['Pred1_sm'])
f1 = scipy.stats.hmean([pr,re])
_min=1000
for i, t in enumerate(th):
    if abs(t-threshold_a)<_min:
        indx = i
        _min = abs(t-threshold_a)
precision_class = pr[indx]
recall_class = re[indx]
f1_class = f1[indx]
auc_class = metrics.roc_auc_score(all_global_SUVR['GT Label'], all_global_SUVR['Pred1'])

precision_regional_global = metrics.precision_score(all_global_SUVR['Abeta GT Positive'], all_global_SUVR['Abeta Pred Positive'])
recall_regional_global = metrics.recall_score(all_global_SUVR['Abeta GT Positive'], all_global_SUVR['Abeta Pred Positive'])
f1_regional_global = metrics.f1_score(all_global_SUVR['Abeta GT Positive'], all_global_SUVR['Abeta Pred Positive'])
auc_regional_global = metrics.roc_auc_score(all_global_SUVR['Abeta GT Positive'], all_global_SUVR['Abeta Pred Positive'])

precision_regional = metrics.precision_score(all_results_df['Abeta GT Positive'], all_results_df['Abeta Pred Positive'])
recall_regional = metrics.recall_score(all_results_df['Abeta GT Positive'], all_results_df['Abeta Pred Positive'])
f1_regional = metrics.f1_score(all_results_df['Abeta GT Positive'], all_results_df['Abeta Pred Positive'])
auc_regional = metrics.roc_auc_score(all_results_df['Abeta GT Positive'], all_results_df['Abeta Pred Positive'])
mse = metrics.mean_squared_error(all_results_df['Abeta GT ROI SUVR'], all_results_df['Abeta Pred ROI SUVR'])

precision_regional_global = format(round(precision_regional_global, 2),'.2f')
recall_regional_global = format(round(recall_regional_global, 2),'.2f')
f1_regional_global = format(round(f1_regional_global, 2),'.2f')
auc_regional_global = format(round(auc_regional_global, 2),'.2f')

precision_regional = format(round(precision_regional, 2),'.2f')
recall_regional = format(round(recall_regional, 2),'.2f')
f1_regional = format(round(f1_regional, 2),'.2f')
auc_regional = format(round(auc_regional, 2),'.2f')
mse = format(round(mse, 3),'.3f')

precision_class = format(round(precision_class, 2),'.2f')
recall_class = format(round(recall_class, 2),'.2f')
f1_class = format(round(f1_class, 2),'.2f')
auc_class = format(round(auc_class, 2),'.2f')

print('Precision: (Regional) {} | (Regional Global) {} | (Classification) {}'.format(precision_regional, precision_regional_global, precision_class))
print('Recall:    (Regional) {} | (Regional Global) {} | (Classification) {}'.format(recall_regional, recall_regional_global, recall_class))
print('F1:        (Regional) {} | (Regional Global) {} | (Classification) {}'.format(f1_regional, f1_regional_global, f1_class))
print('AUC:       (Regional) {} | (Regional Global) {} | (Classification) {}'.format(auc_regional, auc_regional_global, auc_class))
print('\nMSE: {}\n'.format(mse))
x = len(all_results_df.loc[(all_results_df['{} GT Positive'.format(target)]==1)&(all_results_df['{} Pred Positive'.format(target)]==1)&(all_results_df['{} GT Positive'.format(nontarget)]==0)])
y = len(all_results_df.loc[(all_results_df['{} GT Positive'.format(target)]==1)&(all_results_df['{} Pred Positive'.format(target)]==0)&(all_results_df['{} GT Positive'.format(nontarget)]==0)])
print('Out of {} instances, {} found and {} missed. Thats {}%'.format(x+y,x,y,x/(x+y)*100))

In [ ]:
all_global_SUVR[['Abeta GT Positive','Abeta Pred ROI SUVR']].to_csv('../data/mlp_amyloid.csv')

### GM Density

In [ ]:
name = 'roi_predictor/csv/multitask/gmvolume/version_7'
target = 'gmvolume'
nontarget = 'Tau'
splits = 'splits_volume_ATN_schaefer_multitask'
all_results_df = pd.DataFrame(columns = ['gmvolume GT ROI SUVR','gmvolume Pred ROI SUVR','Tau GT ROI SUVR'])

gt_hip_vols = []
pred_hip_vols = []
all_gt_labels = []
all_pred0 = []
all_pred1 = []

roi_names = ['GM Volume ROI {}'.format(x) for x in commonLabels]

for i in range(5,6):
    gt = 'split_{}'.format(i)
    val_f = f'{BASE}/data/{splits}/{gt}.csv' # MAKE SURE CORRECT SPLIT
    preds_f = f'{BASE}/tb_logs/{name}/{gt}/preds.csv'
    

    header_gt = ['Female','Male','Age','33','34','44','24','23','22']
    header_gt.extend(['Tau ROI {}'.format(x) for x in range(1,201)])
    #header_gt.extend(['Tau ROI {}'.format(x) for x in dktLabels])
    header_gt.extend(roi_names)
    header_gt.append('N+')
    
    header_preds = ['Pred 0','Pred 1']
    header_preds.extend(roi_names)

    gt_df = pd.read_csv(val_f, names=header_gt)
    pred_df = pd.read_csv(preds_f, names=header_preds)
    
    #gt_df = gt_df[(gt_df.iloc[:,0]=='GM Volume ROI 4101')|(gt_df.iloc[:,0]=='GM Volume ROI 4102')]
    #pred_df = pred_df[(pred_df.iloc[:,0]=='GM Volume ROI 4101')|(pred_df.iloc[:,0]=='GM Volume ROI 4102')]
    
    gt_df['Hip Volume'] = gt_df.apply(lambda row: calc_total_vol(row),axis=1)
    pred_df['Hip Volume'] = pred_df.apply(lambda row: calc_total_vol(row),axis=1)
    gt_hip_vols.extend(gt_df['Hip Volume'])
    pred_hip_vols.extend(pred_df['Hip Volume'])
    
    all_gt_labels.extend(gt_df['N+'])
    all_pred0.extend(pred_df['Pred 0'])
    all_pred1.extend(pred_df['Pred 1'])
    

    gt_roi_melt_df = pd.melt(gt_df, value_vars = roi_names, value_name='gmvolume GT ROI SUVR')
    pred_roi_melt_df = pd.melt(pred_df, value_vars = roi_names, value_name='gmvolume Pred ROI SUVR')
    results_df = pd.concat([gt_roi_melt_df,pred_roi_melt_df],axis=1)
    all_results_df = pd.concat([all_results_df,results_df[['gmvolume GT ROI SUVR','gmvolume Pred ROI SUVR']]])

    plotAllPred_density(results_df, target)
    outp = os.path.join(os.path.dirname(preds_f),'density.jpg')
    plt.savefig(outp)
    
all_global_values = pd.DataFrame({'GT Hip Volume':gt_hip_vols, 'Pred Hip Volume':pred_hip_vols, 'GT Labels':all_gt_labels, 'Pred0':all_pred0, 'Pred1':all_pred1})
gt_df_gmdensity = gt_df.copy(deep=True)

In [ ]:
from scipy.stats import pearsonr
#plotAllPred(all_results_df, target)
plotAllPred_density(all_results_df, target)
r = pearsonr(all_results_df['gmvolume Pred ROI SUVR'],all_results_df['gmvolume GT ROI SUVR'])
print(r)
plt.savefig(os.path.join(f'{BASE}/tb_logs',name,'all_splits_density.jpg'))

#### Calc Summary Metrics

In [ ]:
cutoff = 1.6474876557377052 - 1 * 0.08779840023070665 # Hipp only

all_global_values[['Pred0_sm','Pred1_sm']] = all_global_values.apply(lambda row: scipy.special.softmax(row[['Pred0','Pred1']]),axis=1)
all_global_values['Pred Label'] = all_global_values.apply(lambda row: np.argmax(row[['Pred0_sm','Pred1_sm']]),axis=1)

all_global_values['GT Neurodegen Positive'] = all_global_values.apply(lambda row: 1 if row['GT Hip Volume']<cutoff else 0, axis=1)
all_global_values['Pred Neurodegen Positive'] = all_global_values.apply(lambda row: 1 if row['Pred Hip Volume']<cutoff else 0, axis=1)

assert np.all(all_global_values['GT Neurodegen Positive']==all_global_values['GT Labels'])

In [ ]:
# Determine decision cutoff from train data
if True:
    pr, re, th = metrics.precision_recall_curve(all_global_values['GT Labels'],all_global_values['Pred1_sm'])
    f1 = scipy.stats.hmean([pr,re])
    best_i = np.argmax(f1)
    threshold_n = th[best_i]

In [ ]:
cutoff = 1.6474876557377052 - 1 * 0.08779840023070665 # Hipp only

pr, re, th = metrics.precision_recall_curve(all_global_values['GT Labels'],all_global_values['Pred1_sm'])
f1 = scipy.stats.hmean([pr,re])

_min=1000
for i, t in enumerate(th):
    if abs(t-threshold_n)<_min:
        indx = i
        _min = abs(t-threshold_n)
        
precision_class = pr[indx]
recall_class = re[indx]
f1_class = f1[indx]
auc_class = metrics.roc_auc_score(all_global_values['GT Labels'], all_global_values['Pred1'])

precision_regional_global = metrics.precision_score(all_global_values['GT Neurodegen Positive'], all_global_values['Pred Neurodegen Positive'])
recall_regional_global = metrics.recall_score(all_global_values['GT Neurodegen Positive'], all_global_values['Pred Neurodegen Positive'])
f1_regional_global = metrics.f1_score(all_global_values['GT Neurodegen Positive'], all_global_values['Pred Neurodegen Positive'])
auc_regional_global = metrics.roc_auc_score(all_global_values['GT Neurodegen Positive'], all_global_values['Pred Neurodegen Positive'])

mse = metrics.mean_squared_error(all_results_df['gmvolume GT ROI SUVR'], all_results_df['gmvolume Pred ROI SUVR'])

precision_regional_global = format(round(precision_regional_global, 2),'.2f')
recall_regional_global = format(round(recall_regional_global, 2),'.2f')
f1_regional_global = format(round(f1_regional_global, 2),'.2f')
auc_regional_global = format(round(auc_regional_global, 2),'.2f')

mse = format(round(mse, 4),'.4f')

precision_class = format(round(precision_class, 2),'.2f')
recall_class = format(round(recall_class, 2),'.2f')
f1_class = format(round(f1_class, 2),'.2f')
auc_class = format(round(auc_class, 2),'.2f')

print('Precision: (Regional Global) {} | (Classification) {}'.format(precision_regional_global, precision_class))
print('Recall:    (Regional Global) {} | (Classification) {}'.format(recall_regional_global, recall_class))
print('F1:        (Regional Global) {} | (Classification) {}'.format(f1_regional_global, f1_class))
print('AUC:       (Regional Global) {} | (Classification) {}'.format(auc_regional_global, auc_class))
print('\nMSE: {}'.format(mse))

In [ ]:
all_global_values[['GT Neurodegen Positive','Pred1']].to_csv('../data/mlp_neurodegen.csv')

## ATN

In [ ]:
amyloid_overlap_indices = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,54,55,56,57,58,59,60,61,62,63,64,65,66,67,69,70,71,72,73,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,212,213,214]
amyloid_atn_df = all_global_SUVR.iloc[amyloid_overlap_indices].reset_index(drop=True)

In [ ]:
def classify_atn(n, a):
    out = []
    for i in range(len(n)):
        n_ = n[i]
        a_ = a[i]
        if n_ and a_:
            out.append(3) # N+|A+
        elif n_ and not a_:
            out.append(2) # N+|A-
        elif not n_ and a_:
            out.append(1) # N-|A+
        elif not n_ and not a_:
            out.append(0) # N-|A-
    return out

In [ ]:
def majority_vote(preds):
  out = []
  for i in range(len(preds[0])):
    if preds[0][i] == preds[1][i]:
      out.append(preds[0][i])
    elif preds[1][i] == preds[2][i]:
      out.append(preds[1][i])
    else:
      out.append(preds[2][i])
  return out

In [ ]:
gb_pred_neurodegen = [0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0]
lr_pred_neurodegen = [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0]
mlp_pred_neurodegen = [0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0]

maj_vote_n = majority_vote([gb_pred_neurodegen, lr_pred_neurodegen, list(all_global_values['Pred Label'])])
# [0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0]

In [ ]:
gb_pred_amyloid = [0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0]
lr_pred_amyloid = [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0]
mlp_pred_amyloid = [0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0]

maj_vote_n = majority_vote([gb_pred_amyloid, lr_pred_amyloid, list(amyloid_atn_df['Pred Label'])])
# [0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0]

In [ ]:
print(metrics.f1_score(list(all_global_values['GT Labels']),gb_pred_neurodegen))
print(metrics.f1_score(list(all_global_values['GT Labels']),lr_pred_neurodegen))
print(metrics.f1_score(list(all_global_values['GT Labels']),mlp_pred_neurodegen))
print(metrics.f1_score(list(all_global_values['GT Labels']),maj_vote_n))

In [ ]:
gt = classify_atn(all_global_values['GT Neurodegen Positive'],amyloid_atn_df['Abeta GT Positive'])



pred = classify_atn(all_global_values['Pred1_sm']>threshold_n,amyloid_atn_df['Pred1_sm']>threshold_a)
acc = metrics.accuracy_score(gt,pred)
f1 = metrics.f1_score(gt,pred,average='macro')
prec = metrics.precision_score(gt,pred,average='macro')
recall = metrics.recall_score(gt,pred,average='macro')
print('For ATN classification with MLP only, accuracy was {} and F1 was {} (Prec: {} | Rec: {})'.format(acc,f1,prec,recall))

pred = classify_atn(gb_pred_neurodegen,amyloid_atn_df['Pred1_sm']>threshold_a)
acc = metrics.accuracy_score(gt,pred)
f1 = metrics.f1_score(gt,pred,average='macro')
prec = metrics.precision_score(gt,pred,average='macro')
recall = metrics.recall_score(gt,pred,average='macro')
print('For ATN classification with MLP and GB, accuracy was {} and F1 was {} (Prec: {} | Rec: {})'.format(acc,f1,prec,recall))

pred = classify_atn(lr_pred_neurodegen,amyloid_atn_df['Pred1_sm']>threshold_a)
acc = metrics.accuracy_score(gt,pred)
f1 = metrics.f1_score(gt,pred,average='macro')
prec = metrics.precision_score(gt,pred,average='macro')
recall = metrics.recall_score(gt,pred,average='macro')
print('For ATN classification with MLP and LR, accuracy was {} and F1 was {} (Prec: {} | Rec: {})'.format(acc,f1,prec,recall))

pred = classify_atn(maj_vote_n,amyloid_atn_df['Pred1_sm']>threshold_a)
acc = metrics.accuracy_score(gt,pred)
f1 = metrics.f1_score(gt,pred,average='macro')
prec = metrics.precision_score(gt,pred,average='macro')
recall = metrics.recall_score(gt,pred,average='macro')
print('For ATN classification with MLP and majority vote, accuracy was {} and F1 was {} (Prec: {} | Rec: {})'.format(acc,f1,prec,recall))

In [ ]:
correct = 0
miss = 0
double_miss = 0
n_miss = 0
abeta_miss = 0

atn_multiclass_gt = [] # 0=A-/N- | 1=A+/N- | 2=A-/N+ | 3=A+/N+
atn_multiclass_pred = []
for i in range(len(amyloid_atn_df)):
    if amyloid_atn_df.iloc[i]['Abeta GT Positive'] and hip_vols_df.iloc[i]['GT Positive']:
        atn_multiclass_gt.append(0)
    elif amyloid_atn_df.iloc[i]['Abeta GT Positive'] and not hip_vols_df.iloc[i]['GT Positive']:
        atn_multiclass_gt.append(1)
    elif not amyloid_atn_df.iloc[i]['Abeta GT Positive'] and hip_vols_df.iloc[i]['GT Positive']:
        atn_multiclass_gt.append(2)
    else:
        atn_multiclass_gt.append(3)
        
    if amyloid_atn_df.iloc[i]['Abeta Pred Positive'] and hip_vols_df.iloc[i]['Pred Positive']:
        atn_multiclass_pred.append(0)
    elif amyloid_atn_df.iloc[i]['Abeta Pred Positive'] and not hip_vols_df.iloc[i]['Pred Positive']:
        atn_multiclass_pred.append(1)
    elif not amyloid_atn_df.iloc[i]['Abeta Pred Positive'] and hip_vols_df.iloc[i]['Pred Positive']:
        atn_multiclass_pred.append(2)
    else:
        atn_multiclass_pred.append(3)
        
    if False: # Accuracy is bad with unbalanced data
        if (amyloid_atn_df.iloc[i]['Abeta GT Positive']==amyloid_atn_df.iloc[i]['Abeta Pred Positive']) and (hip_vols_df.iloc[i]['GT Positive']==hip_vols_df.iloc[i]['Pred Positive']):
            correct += 1
        elif (amyloid_atn_df.iloc[i]['Abeta GT Positive']!=amyloid_atn_df.iloc[i]['Abeta Pred Positive']) and (hip_vols_df.iloc[i]['GT Positive']!=hip_vols_df.iloc[i]['Pred Positive']):
            double_miss += 1
        else:
            if (amyloid_atn_df.iloc[i]['Abeta GT Positive']!=amyloid_atn_df.iloc[i]['Abeta Pred Positive']):
                abeta_miss += 1
            else:
                n_miss += 1
            miss += 1

precision = metrics.precision_score(atn_multiclass_gt,atn_multiclass_pred,average='macro')
recall = metrics.recall_score(atn_multiclass_gt,atn_multiclass_pred,average='macro')
f1 = metrics.f1_score(atn_multiclass_gt,atn_multiclass_pred,average='macro')
mcc = metrics.matthews_corrcoef(atn_multiclass_gt,atn_multiclass_pred)
auc = metrics.roc_auc_score(atn_multiclass_gt,atn_multiclass_pred,multi_class='ovr')
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1 score is: {}'.format(f1))
print('MCC score is: {}'.format(mcc))
print('AUC score is: {}'.format(auc))
       
            
if False:
    print("Out of {} test sbjects, {} were ATN classified correctly, {} had one predicted biomarker incorrect and {} had both biomarkers incorrect".format(len(hip_vols_df),correct,miss,double_miss))
    print("Abeta miss only: {}".format(abeta_miss))
    print("N miss only: {}".format(n_miss))
    